In [1]:
import pyautogui
import requests
import pandas as pd

In [2]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By

In [3]:
driver = webdriver.Chrome()

In [4]:
driver.get("https://www.disasterassist.gov.au/find-a-disaster/australian-disasters")

In [5]:
scraped_ = []

In [6]:
curpage = driver.find_element(by=By.CLASS_NAME, value="currentPage")

In [7]:
while int(curpage.text) <= 38:
    #print(curpage.text)
    events = driver.find_elements(By.CLASS_NAME, value="desk-result")

    for event in events:
        data = event.find_elements(By.TAG_NAME, "td")

        payload = {}

        payload["start date"] = data[0].text
        payload["end date"] = data[1].text
        payload["state"] = data[2].text
        payload["type"] = data[3].text
        payload["link"] = data[4].find_element(By.TAG_NAME, "a").get_attribute("href")
        payload["title"] = data[4].find_element(By.TAG_NAME, "a").text
        payload["agrn"] = data[5].text

        scraped_.append(payload)

    if int(curpage.text) == 38:
        break
    next_page = driver.find_element(By.ID, "btnNext")
    next_page.click()

In [60]:
df = pd.DataFrame(scraped_)

In [61]:
def get_lga_disaster_data(link):
    page = driver.get(link)

    try:
        container = driver.find_element(By.CLASS_NAME, value="ms-rtestate-field")
        text_content = container.text
    except:
        text_content = "NA"

    try:
        lgas = driver.find_element(By.ID, value="LGA")
        lgas = [lga.text for lga in lgas.find_elements(By.TAG_NAME, "li")]
    except: 
        lgas = ["No listed LGA"]

    return [text_content, lgas]

df["scraped"] = df["link"].apply(get_lga_disaster_data)

In [74]:
df["content"] = df["scraped"].apply(lambda x: x[0])
df["lgas"] = df["scraped"].apply(lambda x: x[1] if len(x) > 0 else ["No listed LGA"])

In [77]:
df["lgas"] = df["lgas"].apply(lambda x: x.append("No listed LGA") if len(x) == 0 else x)

In [82]:
df = df.explode("lgas", ignore_index=True)

In [83]:
df.to_excel("disasterassist_australia.xlsx", index=False)